# Estruturas Criptográficas - Criptografia e Segurança da Informação

## TP1 - Exercício 1

1. Use a package Cryptography   e  o package ascon (instalar daqui) para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data) usando Ascon (ver implementação aqui) em modo de cifra.
    2. As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.
    3. Para implementar a comunicação cliente-servidor use o package python `asyncio`.

## Resolução

Para resolver este exercicio comecamos por instalar e importar os packages necessarios.

Este setup inicial envolveu, para alem do referido no exercicio, a utilizacao do package `nest_asyncio` para garantir a execucao apropriado do `asyncio` no `Jupyter Notebook`. 

In [ ]:
%pip install ascon asyncio nest_asyncio

import ascon
import asyncio
import random
import nest_asyncio

Para continuar o setup do ambiente iniciamos o `nest_asyn` e as variaveis globais para o program.

In [ ]:
nest_asyncio.apply()  
hashlength=16         # Tamanho da hash
tag=0                 # Seq Num da ulima mensagem (legitima) enviada
ext=0                 # Seq Num da ulima mensagem (legitima) recebida

Funcao para cifrar mensagens. Processo:

- Criar a `associated data` para a mensagem especifica
- Criar o `nounce` utilizando `Ascon-XOF`
- Cifrar a mensagem utilizando a chave, o `nounce` e a `associated data`

In [ ]:
def cipher_message(in_message,key):
    global tag
    associated_data=f'''message_{tag}'''.encode()
    tag+=1
    nounce_seed=str(random.getrandbits(128))
    nounce=ascon.hash(nounce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
    try:
        out_message=ascon.encrypt(key, nounce, associated_data, in_message.encode(), variant="Ascon-128")
        print(f"Sending: {in_message}")
        print(f"Outgoing >>> ({out_message},{nounce})")
    except Exception as e:
        print(e)
    return (out_message,nounce)

Funcao para decifrar mensagens. Processo:

- Criar a `associated data` para a mensagem esperada
- Decifrar a mensagem utilizando a chave, o `nounce` recebido e a `associated data`
- Verificar que a mensagem foi decifrada com sucesso

In [ ]:
def read_message(text,key,nounce):
    global ext
    print(f"Incoming <<< ({text},{nounce})")
    associated_data=f'''message_{ext}'''.encode()
    try:
        out_message=ascon.decrypt(key, nounce, associated_data, text, variant="Ascon-128")
    except Exception as e:
        return "[ERROR] Message could not be decrypted"
    if out_message==None:
        return "[ERROR] Message could not be decrypted"
    ext+=1
    return out_message.decode()

Funcao para emitir as mensagens cifradas. Processo:

- Aguarda `input` do utilizador para nova mensagem
- Cifrar a mensagem com recurso a chave e a funcao previamente criada
- Adicionar a mensagem (e o respetivo `nounce`) a queue
- Repetir ate que seja enviado codigo para terminar pelo utilizador 

In [ ]:
async def emitter(queue,key):
    loop = asyncio.get_event_loop()
    while True:
        message=await loop.run_in_executor(None, input, "Message to send > ")
        out=cipher_message(message,key)
        await queue.put(out)
        if message=="exit":
            break

Funcao para receber as mensagens cifradas. Processo:

- Aguarda que uma nova mensagem seja obtida da queue
- Verificar se o `nounce` ja foi recebido previamente
    - Se sim, rejeitar a mensagem
- Decifrar a mensagem com recurso a chave, ao `nounce` e a funcao previamente criada
- Repetir ate que seja recebido um codigo para terminar pelo emissor 

In [ ]:
async def receiver(queue,key):
    known_nounces=[]
    while True:
        message,nounce=await queue.get()
        if nounce in known_nounces:
            print("Repeated nounce, ignoring message")
        else:
            known_nounces.append(nounce)
            text=read_message(message,key,nounce)
            print(f"Received: {text}")
        if text=="exit":
            break

Funcao principal por:

- Criar a queue onde serao transmitidas mensagens
- Criar uma key com recurso a `Ascon-XOF` com base numa seed fornecida pelo utilizador
- Criar e lancar os processos para o emissor e recetor

In [ ]:
async def main():
    queue = asyncio.Queue()
    key_seed=input("Seed for key > ")
    key=ascon.hash(key_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
    print(f"Session Key: {key}")
    # Create separate tasks for emitter and receiver
    emitter_task = asyncio.create_task(emitter(queue, key))
    receiver_task = asyncio.create_task(receiver(queue, key))
    # Wait for both tasks to complete
    await asyncio.gather(emitter_task, receiver_task)
    print("All tasks completed")


Finalmente o programa pode ser testado:

In [ ]:
await main()

### Possiveis Problemas

- Alteracoes a mensagem enviada 
- Retransmissoes de mensagens (mesmo `nounce`)

Para testar estes possiveis problemas foram adicionados os comandos `altered_test` e `repeat_test`, respetivamente, para simular estas atitudes.

In [ ]:
sent_nounces=[]

def cipher_message(in_message,key):
    global tag

    # Message data is altered
    if in_message=="altered_test":
        associated_data=f'''message_altered'''.encode()
    else:
        associated_data=f'''message_{tag}'''.encode()
        tag+=1

    # Message is repeated
    if in_message=="repeat_test":
        tag-=1
        nounce=sent_nounces[-1]
    else:
        nounce_seed=str(random.getrandbits(128))
        nounce=ascon.hash(nounce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)

    try:
        out_message=ascon.encrypt(key, nounce, associated_data, in_message.encode(), variant="Ascon-128")
        print(f"Sending: {in_message}")
        print(f"Outgoing >>> ({out_message},{nounce})")
        sent_nounces.append(nounce)
    except Exception as e:
        print(e)
    
    return (out_message,nounce)

Assim podemos testar estas atitudes iniciando o programa e, apos a primeira mensagem enviar:

- `altered_test` => Para enviar uma mensagem com a `associated_data` alterada
- `repeat_test` => Para enviar uma mensagem com o `nounce` anterior 

In [ ]:
await main()